In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import glob
from datetime import datetime as dt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)

In [15]:
# df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

In [2]:
df_sf_2017 = pd.read_json('df_sf_2017')

In [3]:
df_sf_2017.head()

,access,accommodates,amenities,availability_30,availability_365,availability_60,availability_90,bathrooms,bed_type,bedrooms,beds,calculated_host_listings_count,calendar_last_scraped,calendar_updated,cancellation_policy,city,cleaning_fee,country,country_code,description,experiences_offered,extra_people,first_review,guest_per_bed,guest_per_bedroom,guests_included,has_availability,host_about,host_about_filled,host_acceptance_rate,host_has_profile_pic,host_id,host_identity_verified,host_is_superhost,host_listings_count,host_location,host_name,host_neighbourhood,host_picture_url,host_picture_url_filled,host_response_rate,host_response_time,host_since,host_thumbnail_url,host_total_listings_count,host_url,host_verifications,house_rules,id,instant_bookable,interaction,is_business_travel_ready,is_location_exact,jurisdiction_names,last_review,last_scraped,latitude,license,listing_url,longitude,market,maximum_nights,medium_url,minimum_nights,month,monthly_price,name,neighborhood_overview,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,notes,number_of_reviews,picture_url,popular,price,price_per_bedroom,price_per_guest,property_type,require_guest_phone_verification,require_guest_profile_picture,requires_license,review_scores_accuracy,review_scores_checkin,review_scores_cleanliness,review_scores_communication,review_scores_location,review_scores_rating,review_scores_value,reviews_per_month,room_type,scrape_id,security_deposit,smart_location,space,square_feet,state,street,summary,thumbnail_url,transit,weekly_price,xl_picture_url,year,zipcode
100000,"You have access to the entire place - including the kitchen, living room, dining room, and washer/dryer located in the garage! The bathroom will be shared by us two.",3,"{Internet,""Wireless Internet"",Kitchen,""Pets live on this property"",Dog(s),""Buzzer/wireless intercom"",Heating,Washer,Dryer,""Smoke detector"",""Carbon monoxide detector"",""Fire extinguisher"",Essentials,Shampoo,""Lock on bedroom door"",""24-hour check-in"",Hangers,""Hair dryer"",""Laptop friendly workspace"",""translation missing: en.hosting_amenity_49"",""translation missing: en.hosting_amenity_50""}",0,0,0,0,1.0,Real Bed,1,1,3,1485993600000,4 weeks ago,moderate,San Francisco,$30.00,United States,US,"This room is perfectly located for those who enjoy peaceful environments but want to be close to shopping, restaurants, and nightlife. It is one block to the J train (20m ride to downtown) and 3 blocks to Mission! The apartment is a 2 bedroom, 1 (recently remodeled) bathroom. There's a modern kitchen, dishwasher, and washer/dryer in the building for your use. Best of all - no hills to climb to get here! The apartment feels light and airy; it's minimalist with terrariums and various plants scattered throughout. You have access to the entire place - including the kitchen, living room, dining room, and washer/dryer located in the garage! The bathroom will be shared by us two. I'm happy to help out with anything you need! I have a service dog and request that guests please respect his training, which I can quickly go over :) Noe Valley is my favorite neighborhood in San Francisco, and I would love to share it with you! There is truly a sense of community here - most people like to s...",none,8,1.477354e+12,1.0,1.0,1,None,"East coast native, web developer, dog trainer",True,NaN,t,38307539,t,f,3.0,"San Francisco, California, United States",Tess,Noe Valley,https://a0.muscache.com/im/users/38307539/profile_pic/1436769575/original.jpg?aki_policy=profile_x_medium,True,100%,within an hour,2015-07-13,https://a0.muscache.com/im/users/38307539/profile_pic/1436769575/original.jpg?aki_policy=profile_small,3.0,https://www.airbnb.com/users/show/38307539,"['email', 'phone', 'reviews', 'kba']",None,11971314,f,"I'm happy to help out with anything you need! I have a service dog and request that guests please respect his training, which I can quickly go over :)",None,t,SAN FRANCISCO,1.483402e+12,1485993600000,37.742809,None,https:/

In [8]:
average(df_sf_2017.bathrooms)

NameError: name 'average' is not defined

In [56]:
(df_sf_2017['bedrooms']==0).sum()/len(df_sf_2017)

0.08956673977452

In [67]:
# df_sf_2017[df_sf_2017['bedrooms']==0].head(10)

In [69]:
df_sf_2017.property_type.value_counts()

Apartment             58249
House                 30113
Condominium            7132
Loft                   1468
Other                  1415
Bed & Breakfast         998
Townhouse               777
Boutique hotel          665
Dorm                    630
Timeshare               467
In-law                  442
Guest suite             417
Guesthouse              294
Camper/RV               194
Hostel                  185
Bungalow                153
Cabin                   110
Boat                     73
Treehouse                36
Castle                   33
Villa                    28
Lighthouse               15
Yurt                     14
Serviced apartment       14
Cave                     12
Tent                      9
Earth House               6
Pension (Korea)           3
Casa particular           2
Tipi                      1
Vacation home             1
Name: property_type, dtype: int64

In [75]:
df_sf_2017['bedrooms'].value_counts()

1.0     65120
2.0     17962
0.0      9311
3.0      8234
4.0      2639
5.0       383
6.0        94
7.0        47
15.0       36
9.0        22
10.0       20
8.0         5
Name: bedrooms, dtype: int64

In [80]:
df_sf_2017.beds.value_counts()

1.0     60825
2.0     24919
3.0      9737
4.0      4976
5.0      1963
6.0       668
7.0       252
8.0       129
0.0        66
9.0        52
14.0       49
16.0       46
10.0       40
15.0       36
12.0        9
11.0        7
Name: beds, dtype: int64

In [115]:
df_sf_2017.bedrooms.fillna(value = 1, inplace=True)

In [117]:
df_sf_2017.bedrooms.replace(0, 1, inplace=True) ##

In [118]:
df_sf_2017[['guests_included', 'extra_people', 'price','listing_url','bedrooms','beds','has_availability','price_per_guest']].head(30)

,guests_included,extra_people,price,listing_url,bedrooms,beds,has_availability,price_per_guest
22931,2,30.00,98.0,https://www.airbnb.com/rooms/212783,1.0,2.0,NaN,49.000000
22932,2,40.00,195.0,https://www.airbnb.com/rooms/10944402,1.0,1.0,NaN,97.500000
22933,1,0.00,220.0,https://www.airbnb.com/rooms/2965351,2.0,3.0,NaN,220.000000
22934,2,25.00,85.0,https://www.airbnb.com/rooms/16362304,1.0,1.0,NaN,42.500000
22935,1,0.00,150.0,https://www.airbnb.com/rooms/929796,1.0,1.0,NaN,150.000000
22936,1,0.00,195.0,https://www.airbnb.com/rooms/9904485,1.0,1.0,NaN,195.000000
22937,1,15.00,95.0,https://www.airbnb.com/rooms/1931937,1.0,1.0,NaN,95.000000
22938,1,0.00,550.0,https://www.airbnb.com/rooms/19745324,3.0,3.0,NaN,550.000000
22939,1,0.00,180.0,https://www.airbnb.com/rooms/18396762,1.0,1.0,NaN,180.000000
22940,1,0.00,370.0,https://www.airbnb.com/rooms/11187767,3.0,3.0,NaN,370.000000


In [86]:
df_sf_2017.has_availability.value_counts()

t    24759
Name: has_availability, dtype: int64

In [110]:
df_sf_2017['price'] = df_sf_2017.price.astype(float)

In [111]:
df_sf_2017['price_per_guest'] = df_sf_2017['price']/df_sf_2017['guests_included']

In [112]:
df_sf_2017['price_per_guest'].head(20)

22931      49.0
22932      97.5
22933     220.0
22934      42.5
22935     150.0
22936     195.0
22937      95.0
22938     550.0
22939     180.0
22940     370.0
22941     185.0
22942      84.5
22943      57.5
22944      44.5
22945     195.0
22946      98.0
22947     135.0
22948    1850.0
22949     300.0
22950     120.0
Name: price_per_guest, dtype: float64

## for all csv's in the SF folder, concatenate into one dataframe

In [11]:
files = glob.glob('SF/*.gz')
df_sf = pd.concat([pd.read_csv(fp, parse_dates = ['calendar_last_scraped','last_scraped', 'first_review', 'last_review']) for fp in files], ignore_index=True)

ValueError: No objects to concatenate

## Drop the rows where the number of reviews is greater than 0 and the reviews per month is zero

In [6]:
df_sf = df_sf[(df_sf['reviews_per_month'].notnull()) | ((df_sf['number_of_reviews']== 0) & (df_sf['reviews_per_month'].isnull()))]

In [7]:
len(df_sf)

190942

In [8]:
((df_sf_2017['availability_365']==0) & (df_sf_2017['reviews_per_month']==0)).sum()

NameError: name 'df_sf_2017' is not defined

In [80]:
len(df_sf)

119155

In [81]:
71787/190942

0.37596233411192925

## fill in the reviews per month where 0 with 0

In [97]:
df_sf.reviews_per_month.fillna(value=0, inplace=True)

## year and month column

In [98]:
df_sf['year'] = df_sf.calendar_last_scraped.apply(lambda x: x.year)
df_sf['month'] = df_sf.calendar_last_scraped.apply(lambda x: x.month)

## Fill nulls

In [99]:
df_sf.cancellation_policy.fillna(value='none', inplace=True)
df_sf.cleaning_fee.fillna(value='$0.00', inplace=True)

df_sf.host_is_superhost.fillna(value='f', inplace=True)

df_sf['host_is_superhost'].replace('f', False, inplace=True)
df_sf['host_is_superhost'].replace('t', True, inplace=True)


df_sf.loc[df_sf.host_about.notnull(), 'host_about_filled'] = True
df_sf.loc[df_sf.host_about.isnull(), 'host_about_filled'] = False
df_sf.loc[df_sf.host_picture_url.notnull(), 'host_picture_url_filled'] = True
df_sf.loc[df_sf.host_picture_url.isnull(), 'host_picture_url_filled'] = False

In [107]:
df_sf_2017 = df_sf_2017[(df_sf['availability_365']!=0) & (df_sf['reviews_per_month']!=0)]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [108]:
len(df_sf_2017)

62672

In [101]:
df_sf_2017 = df_sf[df_sf['year']==2017]

In [207]:
df_sf_2017['month'].value_counts()

8     9055
1     9035
10    8933
11    8928
7     8854
6     8799
2     8776
5     8732
3     8719
4     8700
9     8527
12    6898
Name: month, dtype: int64

## 2.54 is the cutoff reviews per month to be in the top 20% of listings for all of SF 

In [9]:
df_sf.loc[df_sf.reviews_per_month >= 2.54, 'popular'] = True
df_sf.loc[df_sf.reviews_per_month < 2.54, 'popular'] = False

In [90]:
df_sf_2017.loc[df_sf_2017.reviews_per_month >= 2.57, 'popular'] = True
df_sf_2017.loc[df_sf_2017.reviews_per_month < 2.57, 'popular'] = False

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [231]:
df_sf_2017.popular.value_counts(1)

False    0.799973
True     0.200027
Name: popular, dtype: float64

In [2]:
df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

## Random Forest

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score
from sklearn.preprocessing import LabelEncoder
import pickle

In [4]:
# df[df.columns[1:]].replace('[\$,]', '', regex=True).astype(float)
df_sf_2017['price'].replace('[\$,]', '', regex=True, inplace=True)
df_sf_2017['extra_people'].replace('[\$,]', '', regex=True, inplace=True)

1    9035
2    8776
3    8719
Name: month, dtype: int64

## Function to predict 2017

In [5]:
scores = np.zeros(shape=(4,9))
model_num = 0
start_month = 1
end_month = 4
columns_to_keep = ['accommodates','bed_type', 'extra_people', 'price','guests_included', 'host_about_filled','host_picture_url_filled', 'cancellation_policy', 'room_type', ]


In [13]:
while end_month <13:
    X_train, y_train, X_test, y_test = split_data(start_month, end_month)
    le = LabelEncoder()
    X_train = transform_data(X_train)
    X_test = transform_data(X_test)
    scores = predict_data(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    start_month += 1
    end_month += 1
    model_num += 1
    print(scores)
    break

1 4 0
[[0.81988506 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.2336272  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.51456311 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.32135123 0.         0.         0.         0.         0.
  0.         0.         0.        ]]


In [8]:
def split_data(start_month, end_month):
    X_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)][columns_to_keep]
    y_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['popular']

    X_test = df_sf_2017[df_sf_2017['month'] == end_month][columns_to_keep]
    y_test = df_sf_2017[df_sf_2017['month'] == end_month]['popular']
    
    return X_train, y_train, X_test, y_test
    

In [12]:
def transform_data(X):
    X['extra_people'].replace('[\=$,]', '', regex=True, inplace=True)
    X['price'].replace('[\=$,]', '', regex=True, inplace=True)
    X['price'].apply(pd.to_numeric).apply(np.log)
    X['bed_type'] = le.fit_transform(X['bed_type'])
    X['cancellation_policy'] = le.fit_transform(X['cancellation_policy'])
    X['room_type'] = le.fit_transform(X['room_type'])
 
    return X

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123456)

def predict_data(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators = 1000, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test)
    scores[0][model_num] = accuracy_score(y_test, predicted)
    scores[1][model_num] = recall_score(y_test, rf.predict(X_test))
    scores[2][model_num] = precision_score(y_test, rf.predict(X_test))
    scores[3][model_num] = f1_score(y_test, rf.predict(X_test))
    return scores